In [1]:
%autosave 0

Autosave disabled


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

2022-11-22 07:26:10.251287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#download model and save it as "clothing-model.h5"

#!wget https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5


In [4]:
#download pants image
#!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

In [5]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [6]:
model = keras.models.load_model('clothing-model.h5')

2022-11-22 07:26:17.307336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
img = load_img('pants.jpg', target_size=(299,299))
x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [8]:
X.shape

(1, 299, 299, 3)

In [9]:
pred = model.predict(X)

1/1 [==============================] - 1s 1s/step


In [10]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [11]:
dict(zip(classes, pred[0]))

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}

## Convert Keras to TF-Lite

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing_file.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/sr/7rmvrs4j3976bgrfblnbz1440000gn/T/tmpyzpat4j0/assets


INFO:tensorflow:Assets written to: /var/folders/sr/7rmvrs4j3976bgrfblnbz1440000gn/T/tmpyzpat4j0/assets
2022-11-22 07:27:02.420193: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-22 07:27:02.420218: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-22 07:27:02.421200: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/sr/7rmvrs4j3976bgrfblnbz1440000gn/T/tmpyzpat4j0
2022-11-22 07:27:02.464195: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-22 07:27:02.464235: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/sr/7rmvrs4j3976bgrfblnbz1440000gn/T/tmpyzpat4j0
2022-11-22 07:27:02.602851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2022-11-22 07:27:02.645989: I tensorflow/cc/saved_model/load

In [13]:
import tensorflow.lite as tflite

In [14]:
#load the model
interpreter = tflite.Interpreter(model_path='clothing_file.tflite')

#load the weights
interpreter.allocate_tensors()

In [15]:
#get the inputs
#the index says which part of the model is the input, where the input should go.
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
input_index = interpreter.get_input_details()[0]['index']

In [17]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [18]:
# we have index 0 again because we have only one output, sometimes we can have more
#it means we have 229 things inside our model and this is the last one.
output_index = interpreter.get_output_details()[0]['index']

In [19]:
#initialize the input of the interpreter with X (pants image)
interpreter.set_tensor(input_index, X)

In [20]:
#invoke all the combinations in the neural network. We got the input data and we passed though all this
#base model, layers of the neural network and we now have the results sitting in the 
#output index sitting waiting for us
interpreter.invoke()

In [21]:
#we get our predictions
preds = interpreter.get_tensor(output_index)

In [22]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [23]:
dict(zip(classes, pred[0]))

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}

## Removing TF dependency 

We won't have dependency on tensorflow, we foound how keras implement this and took this code and use it here.

In [24]:
from PIL import Image

In [25]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

/var/folders/sr/7rmvrs4j3976bgrfblnbz1440000gn/T/ipykernel_1013/2391656872.py:2: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((299, 299), Image.NEAREST)


In [26]:
def preprocess_input(x):
    x/=127.5
    x-=1.
    return x

In [27]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [28]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [29]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [30]:
dict(zip(classes, pred[0]))

{'dress': -1.8682904,
 'hat': -4.7612453,
 'longsleeve': -2.3169832,
 'outwear': -1.0625703,
 'pants': 9.887159,
 'shirt': -2.8124344,
 'shoes': -3.666283,
 'shorts': 3.2003617,
 'skirt': -2.602338,
 't-shirt': -4.835046}

## Simpler way  of doing it with keras image helper

In [34]:
pip install keras-image-helper

In [35]:
#by installing tflite we avoid relying on tensorflow
pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [36]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

ImportError: generic_type: type "InterpreterWrapper" is already registered!

In [ ]:
#load the model
interpreter = tflite.Interpreter(model_path='clothing_file.tflite')

#load the weights
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']

output_index = interpreter.get_output_details()[0]['index']

In [ ]:
#create preprocessor
preprocessor = create_preprocessor('xception', target_size=(299, 299))

#get the data
url = 'http://bit.ly/mlbookcamp-pants'

#we can also load it by path instead by url
X = preprocessor.from_url(url)

In [ ]:
#make predictions
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [ ]:
#post predictions
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, pred[0]))